In [1]:
import funzioni02 as f2
import numpy as np
import datetime
import qiskit.ignis.mitigation.measurement as mc
from qiskit import Aer, assemble, QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, transpile, execute
from qiskit.providers.aer import AerSimulator, QasmSimulator
from qiskit.opflow import Zero, One, I, X, Y, Z
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity
import matplotlib.pyplot as plt

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')

sim_noisy_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))
backend_sim_jakarta = sim_noisy_jakarta 
backend = provider.get_backend('ibmq_jakarta')
backend_sim = Aer.get_backend('qasm_simulator')

# Defining all the quantities

In [10]:
shots=8192
reps=4        # al massimo 10
precision=40
time=np.pi
backend_aus=backend_sim_jakarta
check=["4copy_check", [0,2,4,6]]
evo_type="column_evolution" # "column_evolution" or "complete_evolution" or "trotter_steps"
measure_mitigation = "yes"  #"yes" or "no"
steps=86

# Defining the evolution cirquits

In [11]:
qcs_reps = []

for _ in range(reps):
    qcs, qcs_na = f2.evolution_tomo(type=evo_type, N_steps=steps, tempo=time, precision=precision, initial_state='110', check=check)
    qcs_reps=qcs_reps + qcs
    
    qcs, qcs_na = f2.DD_cirquits(N_steps=steps, tempo=time, precision=precision, initial_state='110', check=check)
    qcs_reps=qcs_reps + qcs
reps=reps*2

In [12]:
U = f2.Trotter_N_approx(steps=steps, tempo=time, precision=precision)
   
q1,q2=f2.semplyfied_gates(U,type="DD",precision=precision)
q1.draw()

┌───┐          ┌───────────────┐         ┌───┐                »
q20373_0: ────┤ H ├───────■──┤ Ry(-0.003127) ├──■──────┤ H ├────────────────»
          ┌───┴───┴────┐┌─┴─┐├───────────────┤┌─┴─┐┌───┴───┴────┐           »
q20373_1: ┤ Rz(1.4608) ├┤ X ├┤ Ry(-0.003127) ├┤ X ├┤ Rz(-1.458) ├───────────»
          └───┬───┬────┘└─░─┘└─────┬───┬─────┘└─░─┘└───┬───┬────┘ ░ ┌───┐ ░ »
q20373_2: ────┤ X ├───────░────────┤ X ├────────░──────┤ X ├──────░─┤ X ├─░─»
              └───┘       ░        └───┘        ░      └───┘      ░ └───┘ ░ »
«                                                          
«q20373_0: ────────────────────────────────────────────────
«                                                          
«q20373_1: ────────────────────────────────────────────────
«          ┌───┐ ░ ┌───┐ ░ ┌───┐ ░ ┌───┐ ░ ┌───┐ ░ ┌───┐ ░ 
«q20373_2: ┤ X ├─░─┤ X ├─░─┤ X ├─░─┤ X ├─░─┤ X ├─░─┤ X ├─░─
«          └───┘ ░ └───┘ ░ └───┘ ░ └───┘ ░ └───┘ ░ └───┘ ░

# Defining the calibration cirquits

In [13]:
qcs_calibs, meas_calibs = f2.calibration_cirquits(evo_type+"_remake", q_anc=check[1], check="yes")
state_lables = f2.bin_list(3+len(check[1]))

# Running all the cirquits

In [ ]:
qcs_calibs[-1].draw(output="mpl")

In [14]:
job_evo=execute(qcs_reps, backend=backend_aus, shots=shots)

job_cal=execute(meas_calibs, backend=backend_aus, shots=shots)
job_cal_our=execute(qcs_calibs, backend=backend_aus, shots=shots)

jobs_evo_res=[]

for i in range(reps):
    job=execute(qcs,backend=backend_sim, shots=10)
    results = job.result()
    for j in range(27):
        results.results[j].data.counts = job_evo.result().get_counts()[i*27+j]

    jobs_evo_res.append(results)

# Making the mitigation and computing the fidelity

In [15]:
meas_fitter = mc.CompleteMeasFitter(job_cal.result(), state_labels=state_lables)
meas_fitter_our = mc.CompleteMeasFitter(job_cal_our.result(), state_labels=state_lables)

target_state = (One^One^Zero).to_matrix()

fids=np.zeros([4])

fids=np.zeros([reps,4])
fids_mean=np.zeros(4)
fids_dev=np.zeros(4)

i=0
for res in jobs_evo_res:

    new_res, new_res_nm = f2.mitigate(res, Measure_Mitig="yes", ancillas_conditions=['0011','1110','1101'], meas_fitter=meas_fitter)
    new_res_our, new_res_nm = f2.mitigate(res, Measure_Mitig="yes", ancillas_conditions=['0011','1110','1101'], meas_fitter=meas_fitter_our)
    new_res_not_mitigated = f2.mitigate(res, Measure_Mitig="no", ancillas_conditions=f2.bin_list(4))

    fids[i,0] = f2.fidelity_count(new_res_not_mitigated, qcs_na, target_state)
    fids[i,1] = f2.fidelity_count(new_res_nm, qcs_na, target_state)
    fids[i,2] = f2.fidelity_count(new_res, qcs_na, target_state)
    fids[i,3] = f2.fidelity_count(new_res_our, qcs_na, target_state)

    i+=1

for i in range(4):
    fids_mean[i]=np.mean(fids[:,i])
    fids_dev[i]=np.std(fids[:,i])

In [16]:
for i in range(4):
    print(fids_mean[i], "+-" ,fids_dev[i])

mean_DD=0
mean_=0
for i in range(reps):
    if i%2==0:
        mean_+=fids[i,3]
    if i%2==1:
        mean_DD+=fids[i,3]
mean_DD=mean_DD/reps*2
mean_=mean_/reps*2

print(mean_)
print(mean_DD)

0.845518371328757 +- 0.001457423511592205
0.8964298212411559 +- 0.0011166288510565557
0.9710122876960703 +- 0.0011552368307052006
0.9778480464144583 +- 0.00137073979935461
0.978636846184956
0.9770592466439606


In [17]:
fids

array([[0.84591833, 0.89627272, 0.9719419 , 0.97915345],
       [0.84675452, 0.89679313, 0.96958935, 0.97601512],
       [0.84504566, 0.89574137, 0.97287384, 0.97986087],
       [0.84550898, 0.89799749, 0.97019403, 0.97722524],
       [0.84809447, 0.8973032 , 0.96954881, 0.97614335],
       [0.84497856, 0.89667873, 0.97084592, 0.97747599],
       [0.84516105, 0.89667223, 0.97222313, 0.97938971],
       [0.8426854 , 0.89397969, 0.97088132, 0.97752064]])